<a href="https://colab.research.google.com/github/ethanduk/Ethan-Duke-Tutorial/blob/main/wordle_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# get word lists using bash commands
!wget https://raw.githubusercontent.com/sf200212345/mdst-cracking-wordle/main/wordle/valid_guesses.txt
!wget https://raw.githubusercontent.com/sf200212345/mdst-cracking-wordle/main/wordle/valid_solutions.txt

--2023-10-08 15:52:52--  https://raw.githubusercontent.com/sf200212345/mdst-cracking-wordle/main/wordle/valid_guesses.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89129 (87K) [text/plain]
Saving to: ‘valid_guesses.txt’

valid_guesses.txt   100%[===================>]  87.04K  --.-KB/s    in 0.003s  

2023-10-08 15:52:52 (33.9 MB/s) - ‘valid_guesses.txt’ saved [89129/89129]

--2023-10-08 15:52:53--  https://raw.githubusercontent.com/sf200212345/mdst-cracking-wordle/main/wordle/valid_solutions.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response

In [ ]:
import pathlib
import random
import string

In [ ]:
# https://gist.github.com/cfreshman/d97dbe7004522f7bc52ed2a6e22e2c04
GUESSES_PATH = pathlib.Path("valid_guesses.txt")

# https://www.kaggle.com/datasets/bcruise/wordle-valid-words
SOLUTIONS_PATH = pathlib.Path("valid_solutions.txt")

WORD_LENGTH = 5
NUM_GUESSES = 6

In [ ]:
# Wordle class: runs all the Wordle games. Has various flags for how the game class should accept input and how it should display output.
class Wordle:
  def __init__(self, input_function=None, verbose=True, stats=True, simulate=0, initial_guesses=[]) -> None:
    # read word lists into memory, ENFORCING ALL WORDS AS CAPITAL LETTERS
    with open(GUESSES_PATH, "r") as read_obj:
      self.valid_guesses = read_obj.read().split()
      for i in range(len(self.valid_guesses)):
        self.valid_guesses[i] = self.valid_guesses[i].upper()
    with open(SOLUTIONS_PATH, "r") as read_obj:
      self.valid_solutions = read_obj.read().split()
      for i in range(len(self.valid_solutions)):
        self.valid_solutions[i] = self.valid_solutions[i].upper()

    # README
    # keep track of the input function. If you want to accept input from the user, input_function will be of type None
    # all input functions will have a common interface:
    # accepts the arguments: current_guesses, guess_feedback, filtered_guesses, valid_solutions
    # where current_guesses, guess_feedback match their definition down below in the play function
    # and filtered_guesses is the result returned by filter_on_feedback in the Wordle class
    # This function will return a string to be used as the next guess
    self.input_function = input_function

    # verbose = True means print_state will be used after every guess. Otherwise, only the stats are printed.
    self.verbose = verbose

    # stats = True means stats will be printed after every game, otherwise stats will be printed at the end
    self.stats = stats

    # simulate > 0 means you want to simulate the input number of rounds.
    # otherwise the game class will prompt you if you want to continue playing
    self.simulate = simulate

    # keep a list of initial guesses to use, if your algorithm demands it
    # this will be used to initialize current_guesses in the play loop
    self.initial_guesses = initial_guesses
    for i in range(len(self.initial_guesses)):
      self.initial_guesses[i] = self.initial_guesses[i].upper()

    # keep relevant stats
    self.num_games = 0
    self.num_guesses = 0
    self.num_wins = 0

  def play(self) -> None:
    while True:
      # select solution word
      current_solution = random.choice(self.valid_solutions)

      current_guesses = self.initial_guesses.copy()

      # C for correct, M for misplaced, W for wrong
      guess_feedback = []

      # since there are initial guesses now, also need to populate guess_feedback
      for i in current_guesses:
        guess_feedback.append(self.generate_feedback(i, current_solution))

      # start a game
      for guess_num in range(len(current_guesses), NUM_GUESSES):
        if self.input_function is None:
          current_guess = input(f"Guess {guess_num + 1}: ").upper()
        else:
          current_guess = self.input_function(current_guesses,
                                              guess_feedback,
                                              self.filter_on_feedback(current_guesses, guess_feedback),
                                              self.valid_solutions)

        # make sure the user guess is valid
        while not self.is_valid_guess(current_guess):
          if self.input_function is None:
            current_guess = input(f"Invalid guess, try again. Guess {guess_num + 1}: ").upper()
          else:
            # should not ever get to this else statement, since input_function should return a word from the list of valid guesses/solutions
            current_guess = self.input_function(current_guesses,
                                                guess_feedback,
                                                self.filter_on_feedback(current_guesses, guess_feedback),
                                                self.valid_solutions)

        # valid guess has been obtained
        current_guesses.append(current_guess)
        if current_guess == current_solution:
          break

        # generate feedback for current guess
        # change to "C" if correct, change to "M" if misplaced. W is wrong
        guess_feedback.append(self.generate_feedback(current_guess, current_solution))


        # print state after every guess
        if self.verbose:
          self.print_state(current_guesses, guess_feedback)

      # ending states
      if self.verbose:
        if current_guesses[-1] == current_solution:
            print(f"\nCongratulations! You have correctly guessed {current_solution} in {len(current_guesses)} tries!")
        else:
            print(f"\nUnfortunately, you have failed to correctly guess {current_solution}.")

      # update stats and print after each game
      self.get_print_stats(current_guesses[-1] == current_solution, len(current_guesses))

      # let user decide when to quite when simulate == 0
      if self.simulate == 0:
        if input("Enter 'quit' to quit. Enter anything else to continue playing: ").upper() == "QUIT":
            break
      else:
        if self.simulate == self.num_games:
          break

  # checks if a guess is valid. Current guess has to be WORD_LENGTH in length, all letters, and in valid_guesses
  def is_valid_guess(self, current_guess: str) -> bool:
    return len(current_guess) == WORD_LENGTH and current_guess.isalpha() and current_guess.upper() in self.valid_guesses

  # generates feedback for current guess
  # static method so you can call this method with Wordle.generate_feedback outside of the class
  # notice how there is no "self" argument here
  @staticmethod
  def generate_feedback(current_guess, current_solution):
    # using an array because strings are immutable in Python
    feedback = ["W"] * WORD_LENGTH
    # create a dictionary mapping letters to num_occurences for current_solution for words with multiple of the same letter
    # e.g. solution is GROWS, guess is GOOSE, the middle O should have C and the first O should be W
    solution_letters = {}
    for i in current_solution:
      if solution_letters.get(i) is None:
        solution_letters[i] = 1
      else:
        solution_letters[i] += 1

    # need to iterate through the entire word for correct letters first before generating misplaced letters, see example above.
    for i in range(WORD_LENGTH):
      if current_guess[i] == current_solution[i]:
        feedback[i] = "C"
        # modify solution_letters as well to prevent the wrong feedback from being generated, see example above.
        solution_letters[current_guess[i]] -= 1

    for i in range(WORD_LENGTH):
      if current_guess[i] != current_solution[i] and current_guess[i] in current_solution and solution_letters[current_guess[i]] > 0:
        feedback[i] = "M"
        solution_letters[current_guess[i]] -= 1

    # feedback is default initialized to W, so no need to update feedback for this

    return "".join(feedback)

  # takes in the current guesses and guess feedback and returns a filtered list of valid_guesses
  def filter_on_feedback(self, current_guesses, guess_feedback):
    # disregard green tiles, handle that separately. If green tiles are used to filter the list of valid_guesses,
    # guesses may not be great. E.g. GREEN returns CCCMW, if you limit the guessing space to words starting with GRE,
    # it's hard to eliminate other letters as there are only 2 more spots left
    # We will let individual algorithms handle what to do with the green tiles

    filtered_guesses = []

    # keeps an array of all letters that shouldn't be in that position
    # this means either letters that are wrong (these letters should be added to all letter positions)
    # or letters that are misplaced for this position
    letter_not_in_position = [set() for i in range(WORD_LENGTH)]
    # helps in cases where a word has multiple of the same letter
    not_wrong_letters = set()

    # need separate for loops because words with multiple of the same letter would have that letter be propogated to all positions
    for guess, feedback in zip(current_guesses, guess_feedback):
      for i in range(WORD_LENGTH):
        if feedback[i] == "M":
          letter_not_in_position[i].add(guess[i])
          not_wrong_letters.add(guess[i])
        elif feedback[i] == "C":
          # read the description above for why we don't do anything else with correct letters
          not_wrong_letters.add(guess[i])
    for guess, feedback in zip(current_guesses, guess_feedback):
      for i in range(WORD_LENGTH):
        if feedback[i] == "W":
          if guess[i] in not_wrong_letters:
            letter_not_in_position[i].add(guess[i])
          else:
            # propagate to all positions, since letter cannot be in word at all
            for j in range(WORD_LENGTH):
              letter_not_in_position[j].add(guess[i])

    # now go through the list of guesses and append to filtered guesses if no letters in word at that position are in letter_not_in_position
    for word in self.valid_guesses:
      valid = True
      for i in range(WORD_LENGTH):
        if word[i] in letter_not_in_position[i]:
          valid = False
          break
      if valid:
        filtered_guesses.append(word)

    return filtered_guesses

  def get_print_stats(self, win: bool, num_guesses: int) -> None:
    if win:
        self.num_wins += 1
    self.num_games += 1
    self.num_guesses += num_guesses

    if self.stats or (not self.stats and self.simulate > 0 and self.num_games == self.simulate):
      print(f"\nCurrent win rate: {self.num_wins} / {self.num_games} = {self.num_wins / self.num_games}")
      print(f"Average number of guesses: {self.num_guesses / self.num_games}\n")

  def print_state(self, current_guesses: list[str], guess_feedback: list[str]) -> None:
    correct = set()
    misplaced = set()
    wrong = set()
    for i in reversed(range(len(current_guesses))):
      for j in range(WORD_LENGTH):
        if guess_feedback[i][j] == "C":
          correct.add(current_guesses[i][j])

    for i in reversed(range(len(current_guesses))):
      for j in range(WORD_LENGTH):
        if guess_feedback[i][j] == "M" and current_guesses[i][j] not in correct:
          misplaced.add(current_guesses[i][j])
        elif guess_feedback[i][j] == "W":
          wrong.add(current_guesses[i][j])

    print(f"\t\t\tGUESSES\t\t\tFEEDBACK")
    for guess_num in range(NUM_GUESSES):
      if guess_num < len(current_guesses):
          print(f"\t\t\t{current_guesses[guess_num]}\t\t\t{guess_feedback[guess_num]}")
      else:
          print("\t\t\t_____\t\t\t_____")
    print(f"Correct: {', '.join(sorted(list(correct)))}")
    print(f"Misplaced: {', '.join(sorted(list(misplaced)))}")
    print(f"Wrong: {', '.join(sorted(list(wrong)))}")
    print(f"Unused: {', '.join(sorted(list(set(string.ascii_uppercase).difference(correct).difference(misplaced).difference(wrong))))}\n")

In [ ]:
# referred to as the brute force algorithm in week 2 slides
# idea here is to only limit solution space to words that match the feedback pattern of your most recent guess WHEN COMPARED WITH THE SOLUTION
def only_matched_patterns(current_guesses, guess_feedback, filtered_guesses, valid_solutions):
    remaining_solutions = []
    for word in valid_solutions:
      current_word_feedback = Wordle.generate_feedback(current_guesses[-1], word)
      if current_word_feedback == guess_feedback[-1]:
        remaining_solutions.append(word)
    return random.choice(remaining_solutions)

game = Wordle(input_function=only_matched_patterns, verbose=False, stats=False,  simulate=1000, initial_guesses=["CRANE"])
game.play()

KeyboardInterrupt: ignored

In [ ]:
import itertools
from collections import Counter
from math import log2
def get_feedback(guess, solution):
    feedback = [0, 0]  # [correct_position, wrong_position]

    for i, (g, s) in enumerate(zip(guess, solution)):
        if g == s:
            feedback[0] += 3
            print()
        elif g in solution and g != s:
            feedback[1] += 1

    return tuple(feedback)
Correct = Counter(get_feedback("AAAAA","AAAAA"))
print(Correct)

Two_Correct = Counter(get_feedback("TTTMY","LLLLT"))
print(Two_Correct)








Counter({15: 1, 0: 1})
Counter({0: 1, 3: 1})


In [ ]:
def entropy(current_guesses, guess_feedback, filtered_guesses, valid_solutions):
  max_entropy = -1 # Lowest Possible value is 0, so setting it to -1 means any value higher is good guess,
  best_guess = None
  entropy_list = []

  # Loop through all solutions
  for guess in filtered_guesses:
    for solution in valid_solutions:
        feedback = get_feedback(guess, solution)
        entropy_list.append(feedback)
        feedback_counts = Counter(entropy_list)
        total = sum(feedback_counts.values()) # Total information in all possible guesses example 14 bits
        print("Test")
        # Initializing the current_entropy to 0
        current_entropy = 0

        # Iterating through each count in feedback_counts
        for count in feedback_counts.values():
            # Calculating the probability of each feedback
            probability = count / total

            # Calculating the entropy for each feedback and summing them up
            current_entropy -= probability * log2(probability)  # Using -= because the formula includes a negative sign

        if current_entropy > max_entropy:
            max_entropy = current_entropy
            best_guess = guess

    return best_guess
game = Wordle(input_function=only_matched_patterns, verbose=False, stats=True,  simulate=100, initial_guesses=["CRANE"])
game.play()


Current win rate: 1 / 1 = 1.0
Average number of guesses: 4.0


Current win rate: 1 / 2 = 0.5
Average number of guesses: 5.0


Current win rate: 2 / 3 = 0.6666666666666666
Average number of guesses: 4.666666666666667


Current win rate: 3 / 4 = 0.75
Average number of guesses: 5.0


Current win rate: 4 / 5 = 0.8
Average number of guesses: 4.6


Current win rate: 5 / 6 = 0.8333333333333334
Average number of guesses: 4.833333333333333


Current win rate: 5 / 7 = 0.7142857142857143
Average number of guesses: 5.0


Current win rate: 5 / 8 = 0.625
Average number of guesses: 5.125


Current win rate: 6 / 9 = 0.6666666666666666
Average number of guesses: 5.111111111111111


Current win rate: 6 / 10 = 0.6
Average number of guesses: 5.2


Current win rate: 7 / 11 = 0.6363636363636364
Average number of guesses: 5.0


Current win rate: 7 / 12 = 0.5833333333333334
Average number of guesses: 5.083333333333333


Current win rate: 8 / 13 = 0.6153846153846154
Average number of guesses: 5.15384615384615

KeyboardInterrupt: ignored

In [ ]:
# uses letter frequency and feedback weighing as discussed in the week 2 slides
# extended by normalizing with the length of filtered_guesses and the number of occurrences of a letter in a word
# these help to make choosing a value for weighing easier and prevent words with lots of duplicates like EERIE from being favored
def letter_frequency(current_guesses, guess_feedback, filtered_guesses, valid_solutions):
  # build letter frequency dictionary
  frequencies = {}
  for word in filtered_guesses:
    for i in range(WORD_LENGTH):
      if frequencies.get(word[i]) is None:
        frequencies[word[i]] = 1
      else:
        frequencies[word[i]] += 1

  # need to do the bottom two in separate for loops because a letter could not yet be in correct, but will be in the future.
  # if you get to the same letter but it's misplaced, then it will be doubly added to correct and misplaced
  correct = set()
  for i in reversed(range(len(current_guesses))):
    for j in range(WORD_LENGTH):
      if guess_feedback[i][j] == "C":
        correct.add(current_guesses[i][j])

  misplaced = set()
  for i in reversed(range(len(current_guesses))):
    for j in range(WORD_LENGTH):
      if guess_feedback[i][j] == "M" and current_guesses[i][j] not in correct:
        misplaced.add(current_guesses[i][j])

  # generate the sum of letter frequencies for all words left in the solution space
  solution_values = {
      # sum_frequency: [words]
  }

  weighing = 0.5
  for word in filtered_guesses:
    # normalize by # of duplicate letters
    word_letters = {}
    for i in word:
      if word_letters.get(i) is None:
        word_letters[i] = 1
      else:
        word_letters[i] += 1

    current_freq_sum = 0
    for i in range(WORD_LENGTH):
      # normalize the contribution of each letter by # of occurences in the word to prevent options like EERIE
      # also normalize by length of filtered_guesses
      current_freq_sum += frequencies[word[i]] / word_letters[word[i]] / len(filtered_guesses)

      if word[i] in correct:
        current_freq_sum += weighing * 1.5
      elif word[i] in misplaced:
        current_freq_sum += weighing

    if solution_values.get(current_freq_sum) is None:
      solution_values[current_freq_sum] = [word]
    else:
      solution_values[current_freq_sum].append(word)

  # get the words with the same sum of letter frequencies and choose a random one
  best_solutions = solution_values[max([float(i) for i in solution_values.keys()])]
  return random.choice(best_solutions)

game = Wordle(input_function=letter_frequency, verbose=False, stats=False, simulate=1000, initial_guesses=["AUDIO", "SLEPT", "CHARM"])
game.play()

In [ ]:
# to play Wordle from the cli, just run it like this
game = Wordle(initial_guesses=["tommy"],verbose=True)

game.play()